In [1]:
%load_ext rpy2.ipython
import numpy as np
import pandas as pd
import random

In [3]:
parametros = {"perfil_1":{"media_1":50,"desviacion_1":1,
                         "media_2":50,"desviacion_2":1,
                         "media_3":50,"desviacion_3":1},
             "perfil_2":{"media_1":10,"desviacion_1":1,
                         "media_2":10,"desviacion_2":1,
                         "media_3":10,"desviacion_3":1},
              "perfil_3":{"media_1":90,"desviacion_1":1,
                         "media_2":90,"desviacion_2":1,
                         "media_3":90,"desviacion_3":1}
             }
perfiles = ["perfil_1","perfil_2","perfil_3"]
observaciones = []
perfil_obs = []
for idx in range(1000):
    perfil = random.sample(perfiles,1)
    perfil = perfil[0]
    perfil_obs.append(perfil)
    obs_1 = random.normalvariate(parametros[perfil]["media_1"], parametros[perfil]["desviacion_1"])
    obs_2 = random.normalvariate(parametros[perfil]["media_2"], parametros[perfil]["desviacion_2"])
    obs_3 = random.normalvariate(parametros[perfil]["media_3"], parametros[perfil]["desviacion_3"])

    observaciones.append([obs_1, obs_2, obs_3])

def conecta_vecinos_graph(vecinos,observaciones):
    from sklearn.neighbors import NearestNeighbors
    import numpy as np

    X = np.array(observaciones)
    nbrs = NearestNeighbors(n_neighbors=vecinos, algorithm='ball_tree').fit(X)
    distances, indices = nbrs.kneighbors(X)

    vecinos_cercanos_graph = {}
    for index in indices:
        vecinos_cercanos_graph[index[0]] = set(index[1:])

    return vecinos_cercanos_graph

graph_vecinos = conecta_vecinos_graph(5,observaciones)

def bfs(grafo, nodo0):
    visitado, cola = set(), [nodo0]
    while cola:
        vertex = cola.pop(0)
        if vertex not in visitado:
            visitado.add(vertex)
            if vertex not in grafo:
                grafo[vertex] = set()
                #print 'Tupla:', vertex, ',', len(grafo[vertex]), ',', len(visitado)
            cola.extend(grafo[vertex] - visitado)

    return visitado

def searchKNN(grafo, nodo0):
    klts = []
    faltantes = grafo
    visitados = bfs(grafo, nodo0)
    c=0
    while len(faltantes)>0:
        kls = {}

        for visitado in list(visitados):
            if grafo.has_key(visitado):
                kls[visitado]= grafo[visitado] #guardamos en el cluster
                del faltantes[visitado]
                visitados.remove(visitado)


        klts.append(kls)


        if len(faltantes) >0:
            #print faltantes.keys()[0]
            #print faltantes
            visitados = bfs(faltantes, faltantes.keys()[0])
    return klts

def create_clusterDict(cluster):
    return dict([('cluster_{0}'.format(clu),clusters[clu]) for clu in range(len(clusters))])

def get_joins(dClustered):
    from itertools import chain
    sso = dClustered.items()
    _presup = [[set(sso[idk][1].keys()).intersection(set(sso[idj][1].keys()))for idj in xrange(idk+1,len(sso)) if idj != idk ] for idk, k in enumerate(sso)]    
    return dict([('cluster_{0}'.format(idp),p) for idp, p in enumerate( list(chain(*_presup))) if p!=set()])
    

def create_adMatrix(clustr_dict):
    ##Numero de clusters, estos vienen en formato de diccionario
    num_clusters = len(clustr_dict)
    
    ##Creamos una matriz de adyacencia con puros ceros
    matrix_inicial = np.zeros((num_clusters, num_clusters))
    names = [cluster for cluster in clustr_dict.keys()]
    matrix_distances = pd.DataFrame(matrix_inicial, index=names, columns=names)
    #matrix_distances
    
    ##Hacemos una comparacion entre cada cluster, para sacar el numero de elementos que comparten, quiza no es lo más
    ##eficiente pero es muy sencillo

    for idx_1 in clustr_dict.keys():
        for idx_2 in clustr_dict.keys():
            if idx_1 != idx_2:
                #Elementos en cada cluster
                elementos_uno = float(len(clustr_dict[idx_1]))
                elementos_dos = float(len(clustr_dict[idx_2]))

                #El cluster con el menor numero de elementos
                minimo_elementos = min(elementos_uno, elementos_dos)

                #cuantos elementos comparten cada cluster
                elementos_compartidos = len(list(set(clustr_dict[idx_1]).intersection(clustr_dict[idx_2])))

                #Asignamos a la matriz la distancia como 1 - num_elemntos_comp / minim_elementos
                matrix_distances[idx_1][idx_2] = 1- (elementos_compartidos/minimo_elementos) 
                #matrix_distances[idx_1][idx_2] = elementos_compartidos
    
    return matrix_distances
    

In [4]:
graph_vecinos = conecta_vecinos_graph(8, observaciones)
clusters=searchKNN(graph_vecinos, 0)
cdict = create_clusterDict(clusters)
joind = get_joins(cdict)
adyM = create_adMatrix(joind)
adyM

,cluster_90,cluster_119,cluster_9,cluster_12,cluster_128,cluster_129,cluster_126,cluster_100,cluster_87,cluster_68,cluster_41,cluster_26,cluster_117,cluster_115,cluster_55,cluster_121,cluster_48,cluster_127,cluster_77,cluster_137
cluster_90,0.000000,1.00,1.000000,1.0,1.000000,1.0,1.00,0.888889,0.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.0,1.0
cluster_119,1.000000,0.00,1.000000,1.0,1.000000,1.0,0.25,1.000000,1.0,1.0,1.0,1.0,0.500000,1.0,1.0,1.0,1.0,1.000000,1.0,1.0
cluster_9,1.000000,1.00,0.000000,0.0,1.000000,1.0,1.00,1.000000,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,0.857143,1.0,1.0
cluster_12,1.000000,1.00,0.000000,0.0,1.000000,1.0,1.00,1.000000,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,0.000000,1.0,1.0
cluster_128,1.000000,1.00,1.000000,1.0,0.000000,1.0,1.00,1.000000,1.0,1.0,1.0,1.0,0.857143,1.0,1.0,0.0,1.0,1.000000,1.0,1.0
cluster_129,1.000000,1.00,1.000000,1.0,1.000000,0.0,1.00,1.000000,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.0,1.0
cluster_126,1.000000,0.25,1.000000,1.0,1.000000,1.0,0.00,1.000000,1.0,1.0,1.0,1.0,0.250000,1.0,1.0,1.0,1.0,1.000000,1.0,1.0
cluster_100,0.888889,1.00,1.000000,1.0,1.000000,1.0,1.00,0.000000,0.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.0,1.0
cluster_87,0.000000,1.00,1.000000,1.0,1.000000,1.0,1.00,0.000000,0.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.0,1.0
cluster_68,1.000000,1.00,1.000000,1.0,1.000000,1.0,1.00,1.000000,1.0,0.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.0,1.0


In [5]:
#tabler = np.array([])
#for i in range(adyM.shape[0]):
#    for j in xrange(i+1, adyM.shape[0]):
#        tabler.append((i,j,np.array(adyM)[i,j]))

#print adyM
import pandas as pd
import re
adf= pd.DataFrame(adyM)
adf['nodoOrigen']= adf.index
adf_melt= pd.melt(adf,id_vars=['nodoOrigen'],value_vars=list(adf.columns.values)[:-1],var_name='nodoDestino', value_name='valor')
adf_melt= adf_melt[0:225]

#adf_melt['val'] = [str(v) for v in adf_melt['valor']]

adf_melt['orig']= [int(re.sub('^.*_','',n)) for n in adf_melt['nodoOrigen']]
adf_melt['dest']= [int(re.sub('^.*_','',n)) for n in adf_melt['nodoDestino']]
adf_proc= adf_melt[['orig','dest','valor']]

print(adf_proc)
#adf.columns
#adyM.columns
#pandas.melt(adyM,id_vars=)
#pd.melt(df, id_vars=['A'], value_vars=['B', 'C'])


     orig  dest     valor
0      90    90  0.000000
1     119    90  1.000000
2       9    90  1.000000
3      12    90  1.000000
4     128    90  1.000000
5     129    90  1.000000
6     126    90  1.000000
7     100    90  0.888889
8      87    90  0.000000
9      68    90  1.000000
10     41    90  1.000000
11     26    90  1.000000
12    117    90  1.000000
13    115    90  1.000000
14     55    90  1.000000
15    121    90  1.000000
16     48    90  1.000000
17    127    90  1.000000
18     77    90  1.000000
19    137    90  1.000000
20     90   119  1.000000
21    119   119  0.000000
22      9   119  1.000000
23     12   119  1.000000
24    128   119  1.000000
25    129   119  1.000000
26    126   119  0.250000
27    100   119  1.000000
28     87   119  1.000000
29     68   119  1.000000
..    ...   ...       ...
195   121    68  1.000000
196    48    68  1.000000
197   127    68  1.000000
198    77    68  1.000000
199   137    68  1.000000
200    90    41  1.000000
201   119   

In [25]:
tbl = np.array(adf_proc)
colnames = adf_proc.columns
#np.array(adf_proc)

In [27]:
linksJSON = [{colnames[0]:idx[0], colnames[1]:idx[1],colnames[2]:idx[2]} for idx in tbl]
     

In [5]:
#import json
#adf_proc.to_dict()
%R X=c(1,4,5,7); sd(X); mean(X)

array([ 4.25])

In [8]:
Z = np.array([1,4,5,10])

%R -i Z mean(Z)
%R -o W W=Z*mean(Z)
W


array([  5.,  20.,  25.,  50.])

In [9]:
W

array([  5.,  20.,  25.,  50.])

In [1]:
import km
mapper = km.KeplerMapper(verbose=1)

In [2]:
#print(np.array(adf_proc))
projected_data=mapper.fit_transform(np.array(adf_proc))
#projected_data=mapper.fit_transform(adf_proc)
#print(projected_data)
#toponet = mapper.map(projected_data, np.array(adf_melt), nr_cubes=10)
toponet = mapper.map(projected_data, np.array(adf_proc), nr_cubes=10)



# Initialize to use t-SNE with 2 components (reduces data to 2 dimensions). Also note high overlap_percentage.
#mapper = km.KeplerMapper(cluster_algorithm=km.cluster.DBSCAN(eps=0.3, min_samples=15), reducer = km.manifold.TSNE(), nr_cubes=35, overlap_perc=0.9, link_local=False, verbose=2)
# Fit and transform data
#projected_data2 = mapper.fit_transform(projected_data)
#complex = mapper.map(projected_data2, dimension_index=[0,1], dimension_name="t-SNE(2) 2D")

NameError: name 'np' is not defined

In [160]:
mapper.visualize(toponet, path_html="make_circles_keplermapper_output.html", 
                 title="make_circles(n_samples=5000, noise=0.03, factor=0.3)")


Wrote d3.js graph to 'make_circles_keplermapper_output.html'



Wrote d3.js graph to 'make_circles_keplermapper_output.html'


In [197]:
print(toponet["nodes"])
print(toponet["links"])
print(toponet["meta"])

defaultdict(<type 'list'>, {'0_0_0.0_[0]_[ 0.]': [0, 2, 10, 14, 15, 18, 19, 22, 28, 29]})
defaultdict(<type 'list'>, {})
sum
